In [79]:
import numpy as np
from matplotlib import pyplot as plt 
import subprocess
import os
from os import listdir
from os.path import isfile, join
import re
import string
import random

In [80]:
def prepend_line(file_name, line):
    """ Insert given string as a new line at the beginning of a file """
    # define name of temporary dummy file
    dummy_file = file_name + '.bak'
    # open original file in read mode and dummy file in write mode
    with open(file_name, 'r') as read_obj, open(dummy_file, 'w+') as write_obj:
        # Write given line to the dummy file
        write_obj.write(line + '\n')
        # Read lines from original file one by one and append them to the dummy file
        for line in read_obj:
            write_obj.write(line)
    # remove original file
    os.remove(file_name)
    # Rename dummy file as the original file
    os.rename(dummy_file, file_name)

In [6]:
size = 1408 * 1
shape = (size,1)
BYTES_PER_PACKET = 1408
shape_global = (size,1)
shape_local = (BYTES_PER_PACKET,1)
recv_data_global = np.empty(shape_global, dtype = np.uint8)
data_partial = np.empty(shape_local, dtype = np.uint8)
num_it = (size // BYTES_PER_PACKET)
udp_message_global = np.random.randint(low=0, high=((2**8)-1), size=shape, dtype=np.uint8)

In [7]:
f = "/home/jonathan/Documents/Chalmers/Year5/DAT480/Lab_Project/test_vec.txt"
np.savetxt(f,udp_message_global,newline="",fmt="%d ")


In [9]:
# Send 64 bytes of text
txt = "Hi my name is Jonathan, what is your name? How long is the txt?"


## Ruleset histogram


In [51]:
path = "/home/jonathan/Documents/Chalmers/Year5/DAT480/Lab_Project/DAT480/Rules_div_by_length/"
r_hashed = open("/home/jonathan/Documents/Chalmers/Year5/DAT480/Lab_Project/pattern_match_snort3_content_hashed.txt")
r_length = open("/home/jonathan/Documents/Chalmers/Year5/DAT480/Lab_Project/pattern_match_snort3_content_length.txt")
r_text = open("/home/jonathan/Documents/Chalmers/Year5/DAT480/Lab_Project/pattern_match_snort3_content.txt")
t_hashed = r_hashed.read().splitlines(False)
t_hashed = np.array(t_hashed, dtype = str)
t_length = r_length.read().splitlines(False)
t_length = np.array(t_length, dtype=int)
t_text = r_text.read().splitlines(False)
t_text = np.array(t_text, dtype=str)
r_hashed.close()
r_length.close()

rules_hashed = np.array([t_length, t_hashed]).T
rules_len_sort = rules_hashed[np.argsort(t_length,axis=0)]
for rule in rules_len_sort:
    length = rule[0]
    f = open(path+"rule_len_"+length+".txt","a+")
    f.write("0x"+rule[1]+",\n")
    f.close()

### Generate header file with all arrays containing rules

In [12]:
#concatenate each file in to one large file
files = [f for f in listdir(path) if isfile(join(path, f))]
os.chdir(path)

for file in files:
    fl = (re.search("[0-9]{1,3}",str(file)).group(0))
    prepend_line(path+file, "uint32_t rule_"+fl+" [] = {\n")
    f=open(path+file,"a+")
    f.write("};\n")
    f.close()
    
with open('total_ruleset.h', 'w') as outfile:
    for fname in files:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

AttributeError: 'NoneType' object has no attribute 'group'

In [81]:
arr = (np.array(np.unique(rules_len_sort[:,0],return_counts=True),dtype=int))
indices=np.argsort(arr[0])

sorted_arr = np.array([arr[0][indices], arr[1][indices]])
#size = number of unique lengths, number of maximum elements for any length
two_d_ruleset = np.zeros([len(np.unique(rules_len_sort[:,0])),sorted_arr[1].max()],dtype=int)
prev_length = 0
n_idx = 0
idx = -1
# create 2D array of all hashes, row corresponds to length, column to rule
for _, rule in enumerate(rules_len_sort):
    length = int(rule[0])
    if(length != prev_length):
        prev_length = length
        n_idx = 0
        idx+=1
    two_d_ruleset[idx][n_idx] = int(rule[1],16)
    n_idx += 1

#write the number of elements per unique length in the order of the lengths
f = open(path+"elements.h","w+")
f.write("uint16_t elements [] = {\n")
for k in range(len(sorted_arr[0])):
    f.write(str(sorted_arr[1][k])+",\n")
f.write("};")

#write the unique lengths in order
f = open(path+"lengths.h","w+")
f.write("uint16_t lengths [] = {\n")
for k in range(len(sorted_arr[0])):
    f.write(str(sorted_arr[0][k])+",\n")
f.write("};")
    
# write the 2d array conatining all the rules
#f = open(path+"rules.h","w+")
#f.write("uint32_t rules["+str(len(two_d_ruleset))+"]["+str(len(two_d_ruleset[0]))+"] = {\n")
#for i in range(len(two_d_ruleset)):
#    f.write("{")
#    for j in range(len(two_d_ruleset[0])):
#        f.write(hex(two_d_ruleset[i][j])+", ")
#    f.write("},\n")
#f.write("};")
#f.close()



## Generate input reference and golden reference

In [158]:
number_of_rules_to_test = 1000
number = 0
hash_bin = "/home/jonathan/Documents/Chalmers/Year5/DAT480/Prototyping/CRC/hash.o"
input_vec = open(path+"test_vec_input.txt","w+")
gold_vec = open(path+"test_vec_gold.txt","w+")
for i in range(1,number_of_rules_to_test):
    if(i%2):
        length = random.randrange(1, 365)
        randomstr = ''.join(random.choices(string.ascii_letters+string.digits,k=length))
        args = (hash_bin, randomstr)
        popen = subprocess.Popen(args, stdout=subprocess.PIPE)
        popen.wait()
        output = popen.stdout.read().decode().strip("\n")
        input_vec.write(randomstr)

    else:
        number = random.randrange(1,2677)
        text = t_text[number]
        args = (hash_bin, text)
        popen = subprocess.Popen(args, stdout=subprocess.PIPE)
        popen.wait()
        output = popen.stdout.read().decode().strip("\n")
        gold_vec.write(str(np.where(rules_len_sort[:,1] == (np.char.lower(output)))[0][0])+"\n")
        input_vec.write(text)
        
input_vec.close()
gold_vec.close()

In [146]:
np.where(rules_len_sort[:,1] == "c77c1c6a")[0][0]

13